In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

# Preliminary functions
Functions that make our life easier, since we will be reusing some code.

In [2]:
def dimensions(molecule):
    """
    Return the dimensions (N, M) of a molecule in a given basis. Here, N is the number of electrons and M the number of spinors.
    """
    N = molecule.numberOfElectrons()

    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    M = spin_orbital_basis.numberOfSpinors()
    
    return (N, M)

In [3]:
def onv_bases(N, K):
    """
    Create three ONV bases: I for the (N-1) electron system, J for the N-electron system and K for the (N+1)-electron system.
    """

    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2
    
    onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta - 1)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
    onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
    onv_basis_K = gqcpy.SpinResolvedONVBasis(K, N_alpha + 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

    return (onv_basis_I, onv_basis_J, onv_basis_K)

In [4]:
def C_RHF(molecule):
    """
    Generate restricted spinors, defined as the expansion coefficients in the scalar AO basis, from a restricted
    Hartree-Fock calculation.
    """
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    #print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [5]:
def C_UHF(molecule):
    """
    Generate unrestricted spin-orbitals, for each spin component defined as the expansion coefficients 
    in the scalar AO basis, from an unrestricted Hartree-Fock calculation.
    """
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    #print("N_alpha: {}\nN_beta: {}\nK: {}".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    return uhf_parameters.expansion()

In [6]:
def TM(C_ion, C_neutral):
    """
    Return a transformation matrix that expresses a neutral spinor/spin-orbital in terms of the ones of the ionized molecule.
    """
    return C_ion.inverse().transformed(C_neutral)

In [7]:
def uFCI(molecule, onv_basis):
    """
    FCI calculations for an unrestricted spin-orbital basis.
    """
    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G") # the underlying scalar AO basis is not orthonormal
    spin_orbital_basis.lowdinOrthonormalize() # after using Löwdin's orthonormalizing procedure, the spin orbital basis is now orthonormal
    
    hamiltonian = gqcpy.USQHamiltonian.Molecular(spin_orbital_basis, molecule)
    
    solver = gqcpy.EigenproblemSolver.Dense()
    environment = gqcpy.CIEnvironment.Dense(hamiltonian, onv_basis)

    ci_parameters = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
    
    return ci_parameters

# Molecular set-up

Generate the neutral molecule with N electrons, a positive ion with N-1 electrons (charge +1) and a negative ion with N+1 electrons (charge -1).

In [8]:
#neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , 0)  # create a neutral molecule with N electrons
#positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , +1)  # create a an ion N-1 electrons
#negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , -1)  # create a an ion with N+1 electrons

neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule with N electrons
positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion N-1 electrons
negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , -1)  # create a an ion with N+1 electrons

In [9]:
N, M = dimensions(neutral_molecule)
K = M//2 # number of spinors per spin compound

onv_basis_I, onv_basis_J, onv_basis_K = onv_bases(N, K)

# Fukui matrix in the uncorrelated limit

To generate the transformation matrix [$\mathbf{U}^\pm$](https://gqcg-res.github.io/dariatols_knowdes/fukui-matrix.html), we must combine the coefficient matrix of the neutral molecule and the ion. More details on the transformation of the density matrix is found in the link.

In [10]:
C_min = C_UHF(positive_ion)
C_neutral = C_UHF(neutral_molecule)
C_plus = C_UHF(negative_ion)

U_min = TM(C_min, C_neutral)
U_plus = TM(C_plus, C_neutral)

Because unitary transformations preserve the inner product, only $\mathbf{U}$ should be a unitary matrix. 

In [11]:
print("Is C_min / C_plus unitary? {} / {}".format(C_min.isUnitary(), C_plus.isUnitary()))
print("Is C_neutral unitary? ", C_neutral.isUnitary())
print("Is U_min / U_plus unitary? {} / {}".format(U_min.isUnitary(), U_plus.isUnitary()))

Is C_min / C_plus unitary? False / False
Is C_neutral unitary?  False
Is U_min / U_plus unitary? True / True


Build the linear expansion for a CI wave function in the Hartree-Fock formalism, where all coefficients of a given ONV basis are zero except for the HF determinant.

In [12]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_I)
#print(Psi_I.coefficients())
Psi_J = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_J)
#print(Psi_J.coefficients())
Psi_K = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_K)
#print(Psi_K.coefficients())

print("ONV basis dimensions:\n---------------------\nN-1:\t{}\nN:\t{}\nN+1:\t{}".format(
        len(Psi_I.coefficients()), len(Psi_J.coefficients()), len(Psi_K.coefficients())))

ONV basis dimensions:
---------------------
N-1:	735
N:	441
N+1:	147


To calculate the Fukui matrix, we will need the difference of two density matrices, expressed in the same spin-orbital basis. Therefore, we must transform the density matrix of the ionary species. The density matrix of the neutral species `DM_J` is already in terms of its own spin-orbitals and must therefore not be transformed.

In [13]:
DM_I = Psi_I.calculateSpinResolved1DM()
DM_J = Psi_J.calculateSpinResolved1DM()
DM_K = Psi_K.calculateSpinResolved1DM()

DM_I = DM_I.transformed(U_min)
DM_K = DM_K.transformed(U_plus)

Due to the discontinuity in the energy versus the number of electrons, one must define an upper and lower limit of the Fukui function and accordingly split the Fukui matrix into a positive and negative component. Here, `FM_plus` is $F^+_{pq}$ and `FM_min` is $F^-_{pq}$.

In [14]:
FM_min = DM_J - DM_I
FM_plus = DM_K - DM_J

To assess the quality of the algebraic manipulations ([Bultinck2011](https://pubs.rsc.org/en/content/articlelanding/2011/CP/c0cp02268c#!divAbstract)), we must check the effect of adding or removing an electron on the Fukui matrix. To obtain the $(N+1)$-electron system, we have added an $\alpha$-electron to its spin-orbitals. This means that the $\alpha$ part of $\mathbf{F}^+$ will incorporate an additional electron, which must be reflected in the trace of the Fukui matrix. 

In [15]:
np.trace(FM_plus.alpha)

1.0000000000000069

Additionally, since `Psi_J` and `Psi_K` both have the same number of $\beta$-electrons, the trace of $\mathbf{F}^{+,\beta}$ should be zero.

In [16]:
np.trace(FM_plus.beta)

2.427616755621559e-15

We can work analogously for $\mathbf{F}^-$: `Psi_I` has one less $\beta$ electron than `Psi_J` and consequently, the trace of $F^{-,\beta}$ should be 1

In [17]:
np.trace(FM_min.beta)

0.9999999999999963

whereas the trace of $F^{-,\alpha}$ should be zero now.

In [18]:
np.trace(FM_min.alpha)

-2.5431046157819992e-15

### $F^{+}$

First, we will diagonalize $F^{+,\alpha}$ which has trace 1. According to [Bultinck2011](https://pubs.rsc.org/en/content/articlelanding/2011/CP/c0cp02268c#!divAbstract), the Frontier Molecular Orbital (FMO) approximation is perfect if a single eigenvalue equal to 1 appears. Its corresponding Fukui natural should be solely composed of the FMO of the neutral compound. In our case, we have an eigenvalue of 1, with a corresponding Fukui natural that is exclusively constructed of the FMO of the neutral molecule. Other eigenvalues come in pairs, as described in the article.

In [19]:
F_plus_eigenvalues, F_plus_naturals = np.linalg.eigh(FM_plus.alpha)

print(F_plus_eigenvalues)
print("\n\n")
print(np.round(F_plus_naturals, 3))

[-1.32857404e-01  3.74797419e-16  4.33659659e-16  1.39988019e-15
  1.63580995e-15  1.32857404e-01  1.00000000e+00]



[[-0.    -0.997 -0.008 -0.062  0.054  0.    -0.   ]
 [ 0.     0.063 -0.    -0.998  0.009 -0.     0.   ]
 [-0.753 -0.     0.     0.    -0.    -0.658 -0.   ]
 [-0.     0.054 -0.005  0.012  0.998  0.     0.   ]
 [-0.    -0.007  1.    -0.001  0.006  0.    -0.   ]
 [-0.     0.    -0.    -0.     0.     0.    -1.   ]
 [ 0.658 -0.     0.     0.     0.    -0.753 -0.   ]]


Diagonalizing $F^{+,\beta}$ results in eigenvalues that either come in pairs, or are zero.

In [20]:
F_plus_eigenvalues, F_plus_naturals = np.linalg.eigh(FM_plus.beta)

print(F_plus_eigenvalues)
print("\n\n")
print(np.round(F_plus_naturals, 3))

[-8.47147881e-02 -5.32910853e-03 -6.54570832e-16  4.65957588e-20
  1.33891934e-15  5.32910853e-03  8.47147881e-02]



[[-0.005  0.     1.    -0.     0.019 -0.    -0.004]
 [ 0.551 -0.     0.017  0.    -0.664 -0.     0.506]
 [ 0.    -0.709 -0.     0.     0.    -0.705  0.   ]
 [-0.489 -0.     0.01   0.    -0.748 -0.    -0.449]
 [ 0.    -0.    -0.    -1.    -0.    -0.    -0.   ]
 [-0.676  0.     0.    -0.     0.    -0.     0.736]
 [-0.    -0.705  0.     0.     0.     0.709  0.   ]]


### $F^{-}$

Similar to $F^{+}$, we can both diagonalize the $\alpha$ and $\beta$ part of $F^{-}$. $F^{-,\beta}$ with trace 1 will have a Fukui eigenvalue of 1, with a corresponding Fukui natural consisting of the FMO.

In [21]:
F_min_eigenvalues, F_min_naturals = np.linalg.eigh(FM_min.beta)

print(F_min_eigenvalues)
print("\n\n")
print(np.round(F_min_naturals, 3))

[-1.18219603e-01 -9.83592966e-02 -8.47515873e-16 -2.93218051e-16
  9.83592966e-02  1.18219603e-01  1.00000000e+00]



[[-0.004  0.     0.089 -0.996  0.     0.004  0.   ]
 [-0.134  0.    -0.976 -0.086  0.     0.151 -0.   ]
 [ 0.     0.671  0.     0.     0.741 -0.     0.   ]
 [ 0.65  -0.    -0.2   -0.024 -0.    -0.732 -0.   ]
 [-0.     0.     0.    -0.     0.    -0.    -1.   ]
 [ 0.748 -0.     0.     0.     0.     0.664 -0.   ]
 [-0.    -0.741 -0.    -0.     0.671 -0.     0.   ]]


Diagonalizing $F^{-,\alpha}$ results in eigenvalues that either come in pairs, or are zero.

In [22]:
F_min_eigenvalues, F_min_naturals = np.linalg.eigh(FM_min.alpha)
print(F_min_eigenvalues)
print("\n\n")
print(np.round(F_min_naturals, 3))

[-1.82838831e-01 -1.76256683e-01 -7.39123799e-16 -3.85004198e-16
  2.71846167e-20  1.76256683e-01  1.82838831e-01]



[[ 0.     0.    -0.052 -0.999 -0.    -0.    -0.   ]
 [-0.291 -0.     0.889 -0.046 -0.005  0.     0.35 ]
 [ 0.    -0.642 -0.     0.     0.     0.767 -0.   ]
 [ 0.569  0.     0.454 -0.023 -0.003 -0.    -0.685]
 [ 0.    -0.    -0.006  0.001 -1.     0.    -0.   ]
 [ 0.769  0.    -0.     0.     0.     0.     0.639]
 [-0.     0.767  0.    -0.     0.     0.642 -0.   ]]


# Fukui matrix when correlation is present

In [23]:
Psi_I = uFCI(positive_ion, onv_basis_I)
#print(Psi_I.coefficients())
Psi_J = uFCI(neutral_molecule, onv_basis_J)
#print(Psi_J.coefficients())
Psi_K = uFCI(negative_ion, onv_basis_K)
#print(Psi_K.coefficients())

print("ONV basis dimensions:\n---------------------\nN-1:\t{}\nN:\t{}\nN+1:\t{}".format(
        len(Psi_I.coefficients()), len(Psi_J.coefficients()), len(Psi_K.coefficients())))

ONV basis dimensions:
---------------------
N-1:	735
N:	441
N+1:	147


To calculate the Fukui matrix, we will need the difference of two density matrices, expressed in the same spin-orbital basis. Therefore, we must transform the density matrix of the ionary species. The density matrix of the neutral species `DM_J` is already in terms of its own spin-orbitals and must therefore not be transformed.

In [24]:
DM_I = Psi_I.calculateSpinResolved1DM()
DM_J = Psi_J.calculateSpinResolved1DM()
DM_K = Psi_K.calculateSpinResolved1DM()

DM_I = DM_I.transformed(U_min)
DM_K = DM_K.transformed(U_plus)

Due to the discontinuity in the energy versus the number of electrons, one must define an upper and lower limit of the Fukui function and accordingly split the Fukui matrix into a positive and negative component. Here, `FM_plus` is $F^+_{pq}$ and `FM_min` is $F^-_{pq}$.

In [25]:
FM_min = DM_J - DM_I
FM_plus = DM_K - DM_J

To assess the quality of the algebraic manipulations ([Bultinck2012](https://pubs.rsc.org/en/Content/ArticleLanding/CP/2012/C2CP…)), we must check the effect of adding or removing an electron on the Fukui matrix. To obtain the $(N+1)$-electron system, we have added an $\alpha$-electron to its spin-orbitals. This means that the $\alpha$ part of $\mathbf{F}^+$ will incorporate an additional electron, which must be reflected in the trace of the Fukui matrix. 

In [26]:
np.trace(FM_plus.alpha)

0.9999999999999869

Additionally, since `Psi_J` and `Psi_K` both have the same number of $\beta$-electrons, the trace of $\mathbf{F}^{+,\beta}$ should be zero.

In [27]:
np.trace(FM_plus.beta)

-1.6542323066914832e-14

We can work analogously for $\mathbf{F}^-$: `Psi_I` has one less $\beta$ electron than `Psi_J` and consequently, the trace of $F^{-,\beta}$ should be 1

In [28]:
np.trace(FM_min.beta)

0.9999999999999681

whereas the trace of $F^{-,\alpha}$ should be zero now.

In [29]:
np.trace(FM_min.alpha)

-3.985700658404312e-14

### $F^{+}$

First, we will diagonalize $F^{+,\alpha}$ which has trace 1. According to [Bultinck2012](https://pubs.rsc.org/en/Content/ArticleLanding/CP/2012/C2CP22543C#!divAbstract), the Frontier Molecular Orbital (FMO) approximation is perfect if a single eigenvalue equal to 1 appears. For correlated wave functions, this eigenvalue of 1 will be replaced by an eigenvalue close to unity. The normalization of the Fukui function causes the pairing of other eigenvalues to be lost since this property is only for idempotent density matrices (single determinant level of theory).

In [30]:
F_plus_eigenvalues, F_plus_naturals = np.linalg.eigh(FM_plus.alpha)

print(F_plus_eigenvalues)
print("\n\n")
print(np.round(F_plus_naturals, 3))

[-6.12084707e-01 -5.75253846e-03  7.83852014e-07  5.29324001e-04
  1.52690243e-02  6.39622216e-01  9.62415898e-01]



[[ 0.015 -0.095  0.994 -0.    -0.017  0.033 -0.032]
 [-0.137  0.876  0.107  0.     0.243 -0.274  0.261]
 [ 0.768  0.252  0.002  0.    -0.333 -0.263 -0.407]
 [-0.179 -0.117  0.01   0.    -0.717 -0.462  0.476]
 [ 0.    -0.     0.     1.     0.     0.    -0.   ]
 [-0.189  0.382  0.003  0.    -0.552  0.706 -0.123]
 [-0.569  0.037 -0.    -0.    -0.101 -0.377 -0.723]]


Diagonalizing $F^{+,\beta}$ results in eigenvalues without any symmetry.

In [31]:
F_plus_eigenvalues, F_plus_naturals = np.linalg.eigh(FM_plus.beta)

print(F_plus_eigenvalues)
print("\n\n")
print(np.round(F_plus_naturals, 3))

[-8.32639854e-02 -3.36238950e-02 -1.53452329e-06  5.29324001e-04
  4.21274185e-03  3.41838442e-02  7.79635049e-02]



[[-0.059 -0.068  0.995 -0.     0.008 -0.03   0.013]
 [ 0.022  0.725  0.06  -0.     0.337  0.52   0.294]
 [-0.274  0.562  0.022  0.    -0.734 -0.247 -0.092]
 [ 0.947  0.181  0.069 -0.    -0.157 -0.098 -0.178]
 [-0.    -0.    -0.    -1.    -0.    -0.     0.   ]
 [-0.017 -0.225  0.016  0.    -0.381  0.794 -0.415]
 [-0.156  0.265  0.012 -0.     0.422 -0.164 -0.837]]


### $F^{-}$

Similar to $F^{+}$, we can both diagonalize the $\alpha$ and $\beta$ part of $F^{-}$. $F^{-,\beta}$ with trace 1 will have a Fukui eigenvalue close to 1, with a corresponding Fukui natural consisting of the FMO.

In [32]:
F_min_eigenvalues, F_min_naturals = np.linalg.eigh(FM_min.beta)

print(F_min_eigenvalues)
print("\n\n")
print(np.round(F_min_naturals, 3))

[-8.14247579e-01 -5.92526562e-01 -4.38523423e-04  1.98237685e-06
  5.92279175e-01  8.20356667e-01  9.94574839e-01]



[[-0.05  -0.015 -0.034  0.997  0.017 -0.05   0.   ]
 [ 0.418  0.124  0.775  0.073 -0.125  0.433 -0.   ]
 [ 0.499 -0.442 -0.379  0.02   0.456  0.452 -0.   ]
 [ 0.408  0.604  0.001 -0.001  0.548 -0.41  -0.   ]
 [-0.     0.     0.     0.    -0.    -0.    -1.   ]
 [-0.571  0.397 -0.013 -0.001  0.422  0.582 -0.   ]
 [-0.287 -0.516  0.504 -0.03   0.546 -0.314 -0.   ]]


Diagonalizing $F^{-,\alpha}$ results in eigenvalues that do not show any symmetry.

In [33]:
F_min_eigenvalues, F_min_naturals = np.linalg.eigh(FM_min.alpha)

print(F_min_eigenvalues)
print("\n\n")
print(np.round(F_min_naturals, 3))

[-8.13354793e-01 -8.12553741e-02 -7.60813011e-03 -1.75241925e-05
  2.14698335e-03  7.98897510e-02  8.20199087e-01]



[[ 0.057  0.01  -0.017  0.993  0.081 -0.003 -0.054]
 [-0.493 -0.441  0.2    0.085 -0.23   0.434  0.524]
 [ 0.027 -0.628 -0.698 -0.01   0.061 -0.33   0.065]
 [-0.634  0.446 -0.237  0.06  -0.357 -0.453  0.077]
 [ 0.144 -0.148 -0.094  0.029 -0.793  0.244 -0.509]
 [ 0.362  0.422 -0.515  0.008 -0.155  0.387  0.502]
 [ 0.446 -0.11   0.376  0.037 -0.395 -0.537  0.449]]
